<img src="img/bigsem.png" width="40%" align="right">
<img src="img/logo_wiwi.png" width="20%" align="left">





<br><br><br><br>

# Dynamic Programming Models in Combinatorial Optimization
**Winter Term 2022/23**


# 3. DP Models within MIP Formulations
<img src="img/decision_analytics_logo.png" width="17%" align="right">


<br>

<br>
<br>

**J-Prof. Dr. Michael Römer |  Decision Analytics Group**
                                                    


In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit, typeof
from typing import NamedTuple, Callable, Any
from dataclasses import dataclass, field
from numba.experimental import jitclass
import networkx as nx
from collections import deque
import gurobipy as gp
from gurobipy import GRB


# Overview
- DP Models / DDs as basis of flow-based MIPs
  - example: Knapsack Problem
- The Multiple Knapsack Problem
  - standard MIP model
  - as DP Model
  - as DP model with multiple flow units
 - More generic DP  models (without stages)
 - Example: Shift Scheduling


## Example: the 0/1 knapsack problem

Given 
- a knapsack with a capacity $W$ 
- and a set of items, each with a weight $w_i$ and a value $p_i$
- determine the the subset of the items to put in the knapsack such that
  - the total value of the items in the knapsack is maximal and
  - the total weight of the items in the knapsack does not exceed $W$

**Example:**

<img src="./img/greedy/07.png" width="20%" align="right">

Assume you are a thief and you are about to steal the three items depicted below from an appartment. However, your backpack can only fit 35 lbs. Which items should you take?



<img src="./img/greedy/08.png" width="40%" align="left ">

In [33]:
class DP(NamedTuple):
    feasible_decisions : Callable
    transition_function : Callable
    cost_function : Callable
    direction : str # 'max' or 'min'

## Generic helper functions to deal with maximization and minimization

In [34]:
@njit 
def better(value1, value2, direction):
    if direction == "min":
        return value1 < value2
    else:
        return value1 > value2

In [35]:
@njit
def best_element_and_value(elements, values, direction):
    if direction == "min":
        best_index = np.argmin(values)
    else:
        best_index = np.argmax(values)
    return elements[best_index], values[best_index]

In [36]:
@njit
def get_n_best_elements_and_values(n, elements, values, direction):
    
    if direction == "min":
        sorted_indexes = np.argsort(values)
    else:
        sorted_indexes = np.argsort(-values)
    return elements[sorted_indexes[:n]], values[sorted_indexes[:n]]


In [37]:
get_n_best_elements_and_values(2, np.array([1,2,3]), np.array([2,3,5]), 'min')

(array([1, 2]), array([2, 3]))

## Example: A DP model for the Knapsack Problem
- given a  knapsack instance with $N$ items with weights $w_k$ and profits $p_k$ (zero-indexed) and capacity $W$ 

- state $x_k$: accumulated weight after adding the first $k-1$ items, $x_0 = 0$
- decision $u_k \in \{0, 1\}$ (0: do not add item $k$ to the knapsack; 1: add item $k$)
- $U_k(x_k) = \begin{cases} 
                \{0,1\} \quad \mathrm{if} \quad x_k + w_k \leq W \\
                \{0 \} \quad \mathrm{else}
\end{cases}$

- $f(x_k, u_k) = x_k + w_k u_k $

- $g(x_k, u_k) = p_k u_k$

We have a maximization-objective:

$$\max_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$

## The Knapsack DP Model in Python

In [38]:
class  KPInstance(NamedTuple):
    values:np.array
    weights:np.array
    capacity:int
    N:int   

@njit
def feasible_decisions_kp(instance, k, acc_weight):    
    if acc_weight + instance.weights[k] <= instance.capacity: return np.array([0,1])
    else: return np.array([0])

@njit
def transition_function_kp(instance, k, acc_weight, put):
    return acc_weight + put*instance.weights[k]

@njit
def cost_function_kp(instance, k, acc_weight, put):
      return put*instance.values[k]

Putting all together, and stating that we have a maximization objective

In [39]:
dp_kp = DP(feasible_decisions_kp, transition_function_kp,  cost_function_kp, "max")

## An instance reader function for the Knapsack Problem

In [135]:

def read_kp_instance(filename, sorted=True):
    weights=[]
    values=[]
    with open(filename) as f: # open the file
        line = f.readline().split()  # split first row
        number_of_items = int(line[0]) # read number of items
        capacity = int(line[1]) # read capacity
        for i in range(number_of_items): # read rows for the items
            line = f.readline().split() # split row
            values.append(int(line[0])) # read value
            weights.append(int(line[1])) # read weight
            
    values = np.array(values)
    weights = np.array(weights)    
    
    
    if sorted:
        sorted_indexes = np.argsort(-1* values/weights)
    values = values[sorted_indexes]
    weights = weights[sorted_indexes]
     
        
    return KPInstance(values, weights, capacity, number_of_items)



In [41]:
#filename = "./../problems/knapsack/instances/knapPI_1_5000_1000_1"
filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)

## Exact Decision Diagrams

- given a DP model, we can view an exact DD as a state-transition-graph, with one exception:
  - we introduce a terminal node that forms the target of all arcs emanating from layer $N-1$
- just as in the DP by reaching algorithm, we can construct the exact DD by 


## A Decision Diagram data structure

We will introduce a class `DecisionDiagram` that represents a DD
- consisting of $N$ + 1 layers indexed from 0 to $N$
    - each layer is a dictionary where the key is a state and the value is a `NodeInfo` object
  - (problem-specific) state values representing the start (source) state and the sink state
  

In [42]:


class DecisionDiagram:        
    def __init__(self, number_of_layers, source_layer, source_state, sink_state, direction = 'max'):
        self.g = nx.MultiDiGraph()
        self.number_of_layers = number_of_layers
        self.layers = [set() for l in range(0, number_of_layers)]
        self.source_state = source_state
        self.sink_state = sink_state
        self.layers[source_layer].add(source_state)
        self.g.add_node((source_layer, source_state), best_dist = 0, best_in_edge = None)
        self.direction = direction
        self.last_exact_layer = source_layer
        



        
        
        
        
        
        

## Building an exact DD by top-down-compilation
- building an exact DD is basically the same as building the DP by reaching: states are "discovered" layer per layer
- by applying the transition function to each feasible decision in each state in the layer under consideration
- in the following algorithm, we store the best distance from the source / root node as well as the preceding node in each node
- this means that the best path in the DD is computed "in passing"

## Building an exact DD by top-down-compilation in Python
- observe: here, we introduce a sink state as a "dummy" state (that is otherwise not reachable)

In [43]:
def build_exact_dd(dp, instance, start_layer, start_state, sink_state):
    
    dd = DecisionDiagram(instance.N+1, start_layer, start_state, sink_state, dp.direction)
    
    state = start_state
    total_cost = 0
    
    for k in range(0,instance.N):
        
        for state in dd.layers[k]:
            decisions = dp.feasible_decisions(instance, k, state)
            
            for decision in decisions:
                if k < instance.N -1: # if we are the final layer, point to the "sink state"
                    next_state = dp.transition_function(instance,k,state, decision)
                else:
                    next_state = sink_state
                
                add_transition_dd(dd, k, state, decision, next_state, dp.cost_function(instance, k, state, decision))
   
    k = instance.N-1
    
    return dd

## Creating new nodes: adding the result of a transition

In [44]:
def add_transition_dd(dd, layer_index, state, decision, result_state, cost):
    
    layer = dd.layers[layer_index]
    result_layer = dd.layers[layer_index+1] 
    
    node = (layer_index, state)
    
    result_node = (layer_index+1, result_state)
 
    result_dist = dd.g.nodes[node]["best_dist"] + cost  

    if result_state not in result_layer:
        result_layer.add(result_state)
        dd.g.add_node(result_node, best_dist=result_dist, best_in_edge=(node,result_node,decision))
        
    elif better(result_dist, dd.g.nodes[result_node]["best_dist"], dd.direction): 
        dd.g.nodes[result_node]["best_dist"] = result_dist
        dd.g.nodes[result_node]["best_in_edge"]=(node,result_node,decision)
    
    dd.g.add_edge(node, result_node, decision, cost=cost)


## Trying it out, and some utility functions:

In [45]:
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)


..getting the best objective

In [46]:
def get_best_objective (dd):
    return dd.g.nodes[(dd.number_of_layers-1,dd.sink_state)]["best_dist"]

In [47]:
get_best_objective (dd)

9147

..getting the best path

In [48]:

def get_best_path(dd):
    
    decisions = []
    
    state =  dd.sink_state
    k = dd.number_of_layers - 1
        
    while k > 0:
       # print(k)
        node,  next_node, decision = dd.g.nodes[(k,state)]["best_in_edge"]
        #print (dd.layers[k+1])
        decisions.append(decision)
        k, state = node
        
        

    return  dd.g.nodes[(dd.number_of_layers-1,dd.sink_state)]["best_dist"], list(reversed(decisions))


In [49]:
get_best_path(dd)[1][:20] ## first 10 nodes

[np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0)]

..getting the number of nodes

In [50]:
def get_number_of_nodes(dd):
    return dd.g.number_of_nodes()


In [81]:
get_number_of_nodes(dd)

80725

## Reducing an exact DD

One of the key ideas from DDs is that very often, a DD can be compressed / reduced by merging nodes 
that 
- do not have identical (top-down) states
- but are nonetheless **equivalent** in the sense that they have the same *completions*, that is, the same set of partial solutions until the end (the solution sets of their tail subproblems are identical)

This type of equivalence can be identified by an upward-pass starting from the bottom layer $N$ to layer $0$

- in each layer $k$, two nodes are equivalent (are in the same equivalence class) if 
  - they have the same set of feasible decisions 
  - these decisions have the same costs
  - the corresponding arcs point to the same set of nodes in the subsequent layer $k+1$
- for each equivalence class, merge all nodes in that class into a single node

**Attention:** The following implementation assumes that the decision costs are state-independent. If the decision costs (the arc costs) are state-dependent, then we need to add a check for identical costs, too

## Implementing the DD reduction

In [75]:
def reduce_exact_dd(dd):
    
    #proceed from the bottom (last layer) to the top
    k = len(dd.layers)-1
    while k > 0:
        
        # a dict with key: decisions and resulting nodes (forming an equivalence class)
        #       and value: list of states falling into that class
        eq_classes = {}
    
        #1. collect equivalence classes and states/nodes in each class
        for state in dd.layers[k]:
            
            out_arc_info = []
            
            for u,v,decision,data in dd.g.out_edges((k,state),keys=True, data=True):
                out_arc_info.append((decision,v,data["cost"]))
                
            eq_class = tuple(sorted(out_arc_info))
            
            if eq_class not in eq_classes:
                eq_classes[eq_class] = [state]                
            else:
                eq_classes[eq_class].append(state)
        
        # 2. merge all states in each class into a single node
        for eq_class, states in eq_classes.items():            
            while len(states) > 1:
                state_remove = states.pop()
                merge_nodes(dd, k, states[0], state_remove)
                
        k=k-1

## Merging two nodes

In [76]:
def merge_nodes(dd, layer_index, state_orig, state_remove):
    
    node_orig = (layer_index, state_orig)
    node_remove = (layer_index, state_remove)
    
    layer = dd.layers[layer_index]

    

    # 1. Keep the best distance to from the source
    if better(dd.g.nodes[node_remove]["best_dist"], dd.g.nodes[node_orig]["best_dist"], dd.direction):        
        (u,v,d) = dd.g.nodes[node_remove]["best_in_edge"]
        dd.g.nodes[node_orig]["best_in_edge"] = (u,node_orig,d)
        dd.g.nodes[node_orig]["best_dist"] = dd.g.nodes[node_remove]["best_dist"]

    
    # update best stuff
    
    for u, v, decision,data in dd.g.out_edges(node_remove, keys=True, data=True):
        if dd.g.nodes[v]["best_in_edge"] == (u,v,decision):
            dd.g.nodes[v]["best_in_edge"] = (node_orig, decision, v)
            

    # 3. redirect the in-arcs from the removed node to the node to be kept
    
    arcs_to_add = []
    for u,v,decision,data in dd.g.in_edges(node_remove, keys = True, data=True):
        arcs_to_add.append((u,node_orig, decision, data))
                           
    dd.g.remove_node(node_remove)
    layer.remove(state_remove)
    
    dd.g.add_edges_from(arcs_to_add)


## Trying it  out

In [82]:
reduce_exact_dd(dd)

print ("reduced nodes", get_number_of_nodes(dd))

reduced nodes 19957


# DP Models within MIPs




## Observation:  DDs and optimal paths

- instead of computing the optimal path "on the fly", we could simply compute the longest path in the DD from the source to the sink (terminal node)
- since we have a directed acyclic graph, this can even be done in linear time
- we can simply use the function `dag_longest_path_length` from NetworkX


In [55]:
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)

nx.dag_longest_path_length(dd.g, weight="cost")



9147

### Optimal Paths using a MIP solver

- as you may know, instead of calling a shortest path algorithm we may just formulate the shortest path problem as a **min-cost-flow problem** that can can, in turn, be solved by a MIP

The mathematical model would look as follows (assuming that we have a circulation arc $e^\mathrm{circ}$ from source to sink node):

- let the graph be defined as $G(N,E)$
- let us denote the flow variables with $x$ (we're back in the MIP world ;-))


$$\min \sum_{e \in E}c_e x_e$$

s.t.
$$
 \sum_{e \in v^-} x_e = \sum_{e \in v^+} x_e \quad \forall v \in N \\
 x_e^\mathrm{circ} = 1 \\
 x_e \in \{0, 1\} \quad \forall e \in E $$

In [102]:
def build_network_flow_component(m, dd, flow_size = 1):    
    
    ## add flow vars    
    edge_to_flow_var = {}  

    for e, data in dd.g.edges.items():
            edge_to_flow_var[e] = m.addVar(vtype=GRB.INTEGER, obj = data["cost"] )
    
    ## if not existing yet, add circulation arc
    if dd.g.out_degree ( (dd.number_of_layers-1, dd.sink_state) ) == 0:  
        dd.g.add_edge( (dd.number_of_layers-1, dd.sink_state),  (0, dd.source_state), key=-1, cost=0)      

    ## add flow var for circulation arc and fix flow
    edge_to_flow_var[((dd.number_of_layers-1, dd.sink_state),  (0, dd.source_state), -1)] = m.addVar(vtype=GRB.INTEGER, lb=flow_size, ub=flow_size)
    
    ## add flow balance constraints
            
    for v in dd.g.nodes():
        m.addConstr(gp.quicksum(edge_to_flow_var[e] for e in dd.g.in_edges(v,keys=True)) == gp.quicksum(edge_to_flow_var[e] for e in dd.g.out_edges(v, keys=True) ))

    return edge_to_flow_var
            
        
    
    

## Optimal Paths using a MIP solver: Trying it Out

Read instance


In [123]:

filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)

Build DD

In [80]:
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)
get_number_of_nodes(dd)

80725

Build MIP model

In [83]:

m = gp.Model("knapsack_dd")
m.ModelSense = GRB.MAXIMIZE
edge_to_flow_var = build_network_flow_component(m, dd)

Solve it

In [84]:
%%time
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 19957 rows, 36234 columns and 72468 nonzeros
Model fingerprint: 0xc8bdd3e0
Variable types: 0 continuous, 36234 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 19957 rows and 36234 columns
Presolve time: 0.21s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.23 seconds (0.27 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 9147 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.147000000000e+03, best bound 9.147000000000e+03, gap 0.0000%
CPU times: total: 234 ms
Wall time: 256 ms


In [71]:
print(m.ObjVal)

9147.0


## Extracting the solution

- to get the solution path, we can just "follow" the flow variables with value 1:

In [85]:
def get_solution_path(edge_to_flow_var, dd):
    
    decisions = []
    
    
    node = (0, dd.source_state)    
    while node[1] != dd.sink_state:
        for node, target_node, decision in dd.g.out_edges(node, keys=True):
            if edge_to_flow_var[node, target_node, decision].x > 0.1:
                decisions.append(decision)                
                node = target_node
                break
                
    return decisions     
 
decisions = get_solution_path(edge_to_flow_var, dd)
decisions[:10]

[np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1)]

- we can then turn this into a "readable solution"

In [ ]:
def get_items_from_path(decisions):
    solution_items =  []
    for i, dec in enumerate(decisions):
        if dec == 1:
            solution_items.append(i)
            
    return solution_items

In [ ]:
get_items_from_path(decisions)

## Of course, we can benefit from the DD reduction!

In [ ]:
reduce_exact_dd(dd)

In [ ]:
get_number_of_nodes(dd)

..once again, building and solving the model:

In [87]:

m = gp.Model("knapsack_dd")
m.ModelSense = GRB.MAXIMIZE

edge_to_flow_var = build_network_flow_component(m, dd)

In [100]:
%%time
m.optimize()



Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 100 columns and 100 nonzeros
Model fingerprint: 0x8ef00fbf
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [9e+00, 1e+03]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]


Presolve removed 1 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 9147 8817 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.147000000000e+03, best bound 9.147000000000e+03, gap 0.0000%
CPU times: total: 31.2 ms
Wall time: 16.1 ms


## An Observation for the 0/1 KP Model

- let us re-examine the size of the DD before and after the reduction

In [136]:

filename = "./../problems/knapsack/instances/knapPI_1_100_1000_1"
kp_instance = read_kp_instance(filename)
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)
print("nodes before reduction", get_number_of_nodes(dd))
reduce_exact_dd(dd)
print("nodes after reduction", get_number_of_nodes(dd))

nodes before reduction 80725
nodes after reduction 19957


**Remember:** when reading the instance, we sorted the items according to value/weight, because this yielded the best result for the greedy heuristic

- when creating an exact DD, is this still the best sorting / variable ordering?

- turns out not: it is much better to sort according to weight in descending order since this leads to a **smaller** network!

In [137]:
sorted_indexes = np.argsort(-1* kp_instance.weights)
kp_instance = KPInstance(kp_instance.values[sorted_indexes], kp_instance.weights[sorted_indexes], kp_instance.capacity, kp_instance.N)

In [138]:
dd = build_exact_dd(dp_kp, kp_instance, 0, 0,-1)
print("nodes before reduction", get_number_of_nodes(dd))
reduce_exact_dd(dd)
print("nodes after reduction", get_number_of_nodes(dd))

nodes before reduction 21618
nodes after reduction 12218


## Is this any better than a standard MIP model?

The standard MIP formulation for the 0/1 KP looks as follows:
- let $I$ be the set of items, and $W$ be the capacity


$$\min \sum_{i \in I}p_i x_i$$

s.t.
$$
 \sum_{i \in I} x_i \leq W \\
 x_i \in \{0, 1\} \quad \forall i \in I $$



##  In Python

In [97]:
m = gp.Model("knapsack")

m.ModelSense = GRB.MAXIMIZE

x = [m.addVar(vtype=GRB.BINARY) for i in range(kp_instance.N)]


m.setObjective(gp.quicksum(kp_instance.values[i] * x[i] for i in range(kp_instance.N)))

m.addConstr(gp.quicksum(kp_instance.weights[i] * x[i] for i in range(kp_instance.N)) <= kp_instance.capacity)

<gurobi.Constr *Awaiting Model Update*>

In [99]:
%%time
m.optimize()
print(m.objVal)
selected = [i for i in range(kp_instance.N) if x[i].x >= 0.99]
print("selected items: {}".format(selected))


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 100 columns and 100 nonzeros
Model fingerprint: 0x8ef00fbf
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [9e+00, 1e+03]
  Objective range  [7e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]


Presolve removed 1 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 9147 8817 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.147000000000e+03, best bound 9.147000000000e+03, gap 0.0000%
9147.0
selected items: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13]
CPU time

.. this was not a big success for DD, acutually... let's see what comes next

## The Multiple Knapsack Problem

- the multiple knapsack problem is an extension of the 0/1 KP where we do not have a single, but multiple knapsacks
- in general, the knapsacks can have different capacities
- **but here**, for the sake of simplicity, we assume that all knapsacks have the same capacity

..let us first define an instance type

In [141]:
class  MKPInstance(NamedTuple):
    values:np.array
    weights:np.array
    capacity:int
    number_of_knapsacks:int
    N:int
        

..and an instance creation function that uses the KP instance format to create an MKP instance 

In [142]:
       
def create_mkp_instance_from_kp(kp_instance, number_of_knapsacks, weight_factor = 0.5):
    
    cap = int(weight_factor *np.sum(kp_instance.weights) / number_of_knapsacks)    
        
    return MKPInstance(kp_instance.values, kp_instance.weights, cap, number_of_knapsacks, kp_instance.N)

In [143]:
mkp_instance = create_mkp_instance_from_kp(kp_instance, 20)

## The Multiple Knapsack Problem: Standard MIP formulation


- let $I$ be the set of items, let $J$ be the set of knapsacks and $W$ be the capacity
- once again, we assume here (as opposed to standard MKP) that $W$ is the same for all $j \in J$


$$\min \sum_{i \in I}\sum_{j \in j}  p_i x_{ij} $$

s.t.
$$
 \sum_{i \in I} x_{ij} \leq W  \quad \forall j \in J \\
  \sum_{j \in J} x_{ij} \leq 1  \quad \forall i \in I \\
 x_{ij} \in \{0, 1\} \quad \forall i \in I, j \in J $$

## The Multiple Knapsack Problem: Solving the standard MIP formulation in Python



In [111]:
m = gp.Model("multiple_knapsack")


x = [[m.addVar(vtype=GRB.BINARY) for k in range(mkp_instance.number_of_knapsacks)] for i in range(mkp_instance.N)]


m.ModelSense = GRB.MAXIMIZE

m.setObjective (gp.quicksum(mkp_instance.values[i] * x[i][k] for i in range(mkp_instance.N)  for k in range(mkp_instance.number_of_knapsacks) ))

for k in range(mkp_instance.number_of_knapsacks):
    
    m.addConstr(gp.quicksum(mkp_instance.weights[i] * x[i][k] for i in range(mkp_instance.N)) <= mkp_instance.capacity)
    
for i in range(mkp_instance.N):
        m.addConstr(gp.quicksum(x[i][k] for k in range(mkp_instance.number_of_knapsacks)) <= 1)

..solving it

In [ ]:
%%time

m.params.TimeLimit = 2*60 #just to make sure, we'r in class!

m.optimize()

In [118]:
print("objective", m.ObjVal, "upper bound", m.ObjBound, f"gap %  {100*(m.ObjBound - m.ObjVal) / m.ObjVal:0.3f}")

objective 40277.0 upper bound 40390.0 gap %  0.281


## A DP Model for the MKP?

How would a DP model look like for the MKP?

You can use this 0/1 KP model as starting point:

- state $x_k$: accumulated weight after adding the first $k-1$ items, $x_0 = 0$
- decision $u_k \in \{0, 1\}$ (0: do not add item $k$ to the knapsack; 1: add item $k$)
- $U_k(x_k) = \begin{cases} 
                \{0,1\} \quad \mathrm{if} \quad x_k + w_k \leq W \\
                \{0 \} \quad \mathrm{else}
\end{cases}$

- $f(x_k, u_k) = x_k + w_k u_k $

- $g(x_k, u_k) = p_k u_k$

We have a maximization-objective:

$$\max_{u_0,..,u_k,..u_{N-1}} \sum_{k=0}^{N-1} g_k(x_k,u_k)$$

## DP Model for the MKP: In Python

In [ ]:
@njit
def feasible_decisions_mkp(instance, k, acc_weight):    
    #if acc_weight + instance.weights[k] <= instance.capacity: return np.array([0,1])
    #else: return np.array([0])
    return

@njit
def transition_function_mkp(instance, k, acc_weight, put):
    #return acc_weight + put*instance.weights[k]
    return

@njit
def cost_function_mkp(instance, k, acc_weight, put):
      #return put*instance.values[k]
    return

In [ ]:
dp_mkp = DP(feasible_decisions_mkp, transition_function_mkp,  cost_function_mkp, "max")

In [ ]:
N = 20
kp_instance_small = KPInstance(kp_instance.values[:N], kp_instance.weights[:N],kp_instance.capacity,  N)
mkp_instance_small = create_mkp_instance_from_kp(kp_instance_small, 4)

In [ ]:
#start_state = 
# sink_state
#dd = build_exact_dd(dp_mkp, mkp_instance_small, 0, start_state, sink_state)

## Key idea: Multiple flow units through a single-knapsack DD


**Observe the following:**
- in the MKP that we consider here (identical capacities), the set of feasible solutions for each knapsack with capacity $W$ is identical to the set of solutions of a single 0/1-KP with capactiy $W$
- each item can only be assigned to a **single knapsack**

Now let us dicuss a **key idea** useful in many cases, not only for 0/1 KP:
- if we are looking for a solution that can be represented as a set of multiple paths within the same network
  - **we can model this as a (multi-unit) flow through a single network**
  - that can be modeled as part of a MIP model#
  - which gives us the opportunity to add **additional constraints** not represented in the network
  

- after solving such a model, we can obtain feasible solutions via **flow decomposition**

## Formulating the MKP using a DD-based flow component

- let $I$ be the set of items, let $J$ be the set of knapsacks and $W$ be the capacity
-  we assume here (as opposed to standard MKP) that $W$ is the same for all $j \in J$

- assume that we have a graph $G=(V,E)$ that is the graph underlying a DD, augmented with a flow circulation arc $e^\mathrm{circ}$

- let $E^i$ be the set of items representing the set edges picking item $i$

Then, we can formulate the MKP as follows:

$$\min \sum_{e \in E} c_e x_e$$

s.t.
$$
 \sum_{e \in v^-} x_e = \sum_{e \in v^+} x_e \quad \forall v \in N \\
 x_e^\mathrm{circ} = |J| \\ 
 \sum_{e \in E^i} x_e \leq 1 \quad \forall i \in I \\ 
 x_e \in \{0, 1\} \quad \forall e \in E $$

..what is "surprising" / different to the standard MKP formulation here?

## Formulating the MKP using a DD-based flow component: In Python
..building reduced DD

In [144]:
dd = build_exact_dd(dp_kp, mkp_instance, 0, 0,-1)
print ("nodes before reduction", get_number_of_nodes(dd))
reduce_exact_dd(dd)
print ("nodes after reduction", get_number_of_nodes(dd))

nodes before reduction 33696
nodes after reduction 20262


..creating the model

In [145]:
m = gp.Model("mk_knapsack_dd")

m.ModelSense = GRB.MAXIMIZE

# network flow component
edge_to_flow_var = build_network_flow_component(m, dd, mkp_instance.number_of_knapsacks)

# create the sets of edges per item
item_variables = [[] for i in range(mkp_instance.N)]
for e, var in edge_to_flow_var.items():    
    if e[2]==1:
        item_variables[e[0][0]].append(var)        

# create the constraints forcing every item to be picked at most onces     
for i, item_vars in enumerate(item_variables): 
    m.addConstr(gp.quicksum(item_var for item_var in item_vars)  <= 1 )                    

..solving the model

In [146]:
%%time
m.params.TimeLimit = 2*60
m.optimize()

print("objective", m.ObjVal, "upper bound", m.ObjBound, f"gap %  {100*(m.ObjBound - m.ObjVal) / m.ObjVal:0.3f}")

Set parameter TimeLimit to value 120
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  120

Optimize a model with 20362 rows, 34325 columns and 82713 nonzeros
Model fingerprint: 0x9b34cb44
Variable types: 0 continuous, 34325 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+03]
  Bounds range     [2e+01, 2e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 11926 rows and 11885 columns
Presolve time: 0.44s
Presolved: 8436 rows, 22440 columns, 59189 nonzeros
Variable types: 0 continuous, 22440 integer (14492 binary)
Found heuristic solution: objective 29853.000000

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27993    4.0433015e+04   2.992172e+02   0.00000

## Obtaining Paths via Flow Decomposition
.. now let us see how to obtain the solution for each of the knapsacks:
- the core idea here is to "walk through the DD" from source to sink $|J|$ times
- we start by initiating a dict stores the remaining flow through each edge
  - initialized by the flow solution
- then we extract one path by following a "non-zero" path from source to sink, reducing the remaining flow on each visited edge
- until there is no more flow

**Observe:**

In general, such a flow decomposition is not unique!


In [147]:
def decompose_flow_to_solution_paths(edge_to_flow_var, dd):
    
    paths = []
    
    ## inoit    
    edge_to_remaining_flow = {}    
    for e, flow_var in edge_to_flow_var.items():
        edge_to_remaining_flow[e]  = round(flow_var.x)        
    
    circ_edge = ((dd.number_of_layers - 1, dd.sink_state), (0,dd.source_state), -1)   
    
    while edge_to_remaining_flow[circ_edge] > 0:

        edge_to_remaining_flow[circ_edge] -= 1
        
        decisions = []
        node = (0, dd.source_state)

        while node[1] != dd.sink_state:
            for node, target_node, decision in dd.g.out_edges(node, keys=True):
                if edge_to_remaining_flow[(node, target_node, decision)]  > 0:
                    decisions.append(decision)
                    edge_to_remaining_flow[(node, target_node, decision)] -= 1  
                    
                    node = target_node 
                    break
                    
        paths.append(decisions)
                
    return paths
    
    


## Obtaining Paths via Flow Decomposition: Trying it out


In [148]:
paths = decompose_flow_to_solution_paths(edge_to_flow_var, dd)

def get_items_from_path(decisions):
    solution_items =  []
    for i, dec in enumerate(decisions):
        if dec == 1:
            solution_items.append(i)
            
    return solution_items

knapsack_items = [ get_items_from_path(path) for path in paths]
knapsack_items

[[43, 57, 73],
 [41, 46, 90, 99],
 [40, 59, 74],
 [39, 44, 96],
 [38, 58, 79],
 [37, 61, 70],
 [35, 64, 72],
 [34, 53, 91],
 [30, 48],
 [28, 65, 84, 89],
 [27, 66, 83, 92],
 [19, 63, 93],
 [17, 60, 97],
 [15, 67, 88],
 [14, 62, 95],
 [11, 78, 82],
 [9, 75, 86],
 [3, 85, 87, 98],
 [2, 77, 94],
 [0, 71]]

## Concluding Remarks

- we created our first DD-based MIPs 
- these MIPs are often large (although the DD reduction generally helps)
- but a key advantage is that they can model the flow corresponding to multiple identical objects (e.g. knapsacks, workers with identical skills) in a single network which typically reduces symmetry
- and: these models usually have a stronger LP relaxation than "standard formulations"

- these DD-based MIPs can be viewed as a special case of state-expanded networks that we will consider next week